In [1]:
import os
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from dotenv import find_dotenv, load_dotenv
import textwrap

load_dotenv(find_dotenv())

False

In [3]:
class YouTubeChat:
    def __init__(self, video_url):
        self.db = self.create_db_from_youtube_video_url(video_url)
        self.chat_history = []

    def create_db_from_youtube_video_url(self, url):
        loader = YoutubeLoader.from_youtube_url(url)
        transcript = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        docs = text_splitter.split_documents(transcript)
        db = FAISS.from_documents(docs, embedding=OpenAIEmbeddings())
        return db

    def get_response_from_query(self, query, k=4):
        docs = self.db.similarity_search(query, k=k)
        docs_page_content = " ".join([d.page_content for d in docs])
        chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

        template = """
            You are a helpful assistant that can answer questions about youtube videos
            based on the video's transcript: {docs}
            
            Only use the factual information from the transcript to answer the question.
            
            If you feel like you don't have enough information to answer the question, say "I don't know".
            
            Your answers should be verbose and detailed. Limit your response in less than 100 words.
        """
        system_message_prompt = SystemMessagePromptTemplate.from_template(template)
        
        # Add previous chat history
        messages = [system_message_prompt] + self.chat_history
        human_message_prompt = HumanMessagePromptTemplate.from_template("Answer the following question: {question}")
        messages.append(human_message_prompt)
        
        chat_prompt = ChatPromptTemplate.from_messages(messages)
        
        chain = LLMChain(llm=chat, prompt=chat_prompt)
        response = chain.run(question=query, docs=docs_page_content)
        response = response.replace("\n", "")
        
        # Store the current query and response in chat history
        self.chat_history.append(HumanMessagePromptTemplate.from_template(query))
        self.chat_history.append(SystemMessagePromptTemplate.from_template(response))
        
        return response, docs
    
    def interactive_chat(self):
        print("Type 'exit' to end the conversation.")
        while True:
            query = input("You: ")
            if query.lower() == 'exit':
                break
            response, _ = self.get_response_from_query(query)
            print("Bot: ", textwrap.fill(response, width=85))
            print("")

In [4]:
video_url = "https://www.youtube.com/watch?v=L_Guz73e6fw"
youtub_chat = YouTubeChat(video_url)
youtub_chat.interactive_chat()

Type 'exit' to end the conversation.
You: Are they talking about microsoft? Respond in one word, either yes or no.
Bot:  Yes.

You: Can you provide more details?
Bot:  Based on the transcript, there is no explicit mention of Microsoft. The conversation
revolves around topics such as the potential existence of AGI (Artificial General
Intelligence), the impact of digital intelligence, biases in AI models, and the
importance of interacting with users and understanding their perspectives. While the
discussion does not directly reference Microsoft, it is possible that the
conversation could be relevant to the company's work in AI and technology. However,
without further context, it is difficult to provide more specific details.

You: exit
